提升方法通过学习多个弱分类器，并将其加权组合成一个强分类器

In [1]:
import numpy as np
from sklearn.ensemble import AdaBoostClassifier

In [2]:
X = np.array([[0,1,3],[0,3,1],[1,2,2],[1,1,3],[1,2,3],[0,1,2],[1,1,2],[1,1,1],[1,3,1],[0,2,1]])
y = np.array([-1,-1,-1,-1,-1,-1,1,1,-1,-1]).reshape([-1,1])
X.shape,y.shape

((10, 3), (10, 1))

In [9]:
def model(X,y,M):
    '''M为提升的次数
       X m*n m为样本数，n为特征数
       y m*1'''
    y_final = np.zeros(y.shape)
    W_m = np.ones([len(X),1])/len(X)
    for i in range(M):
        #print(i)
        y_pred = logis(X,y,W_m)
        e_m = np.dot(W_m.T,y_pred!=y).item()
        if e_m == 0:
            return y_pred
        if e_m >0.5:
            y_pred = -y_pred
            e_m = 1-e_m
        W_m,alpha_m = propagate(X,W_m,y_pred,e_m)
        y_final += alpha_m*y_pred
    return (y_final>0)*2-1
   
def propagate(X,W_m,y_pred,e_m):
    '''
    输入训练集和权重,预测标签和实际标签
    输出下一轮的权重W_mplus和这轮的预测结果级模型权重
    '''
    
    alpha_m = 0.5 * np.log(1/e_m-1)
    Z_m = np.dot(W_m.T,np.exp(-alpha_m*y*y_pred)).item()
    W_mplus = W_m*np.exp(-alpha_m*y*y_pred)
    return W_mplus,alpha_m
    
def sigmoid(z):
    return 1/(1+np.exp(-z))

def forward(X,W,b):
    return sigmoid(X.dot(W)+b)
    
def back_prop(X,y,W,b,W_m):
    m = X.shape[0]
    z = X.dot(W)+b
    a = sigmoid(z)
    dW = np.dot(X.T,(a-y)*W_m)
    db = (a-y)*W_m
    return (dW,db)

def train(X,y,W_m,lr=0.01,iters=10):
    W = np.random.randn(X.shape[1],1)
    b = 0
    count = 0
    while count < iters:
        dW,db = back_prop(X,y,W,b,W_m)
        W -= lr * dW
        b -= lr * db
        count += 1
    return W,b

def predict(X_test,W,b):
    a = forward(X_test,W,b)
    return np.int64(a>0.5)*2-1

def logis(X,y,W_m,lr=0.1,iters=20):
    W,b = train(X,y,W_m)
    return predict(X,W,b)

In [14]:
model(X,y,200).T

array([[-1, -1, -1, -1, -1, -1,  1,  1, -1, -1]], dtype=int64)

In [12]:
AB = AdaBoostClassifier()
AB.fit(X,y)
AB.predict(X)

C:\Users\Ni\Anaconda3\envs\cs231n\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([-1, -1, -1, -1, -1, -1,  1,  1, -1, -1])